In [1]:
import pandas as pd
data = pd.read_csv('E:\\pyrhon\\progectTEST\\titanic\\train.csv')

In [2]:
from IPython.display import Image

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
data= data.drop(['Name', 'Ticket', 'Cabin','PassengerId'], axis =1)

Перекодировка пола

In [7]:
data['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [8]:
sex_mapping = {'male':0, 'female':1}
data['Sex'] = data['Sex'].map(sex_mapping)

Теперь нужно перкодировать Embarked, там 3 класса, нужно перекодировать по другому.S - 001, C - 100, Q- 010
делается это функцией get_dumeies

In [9]:
Embarked_dummies = pd.get_dummies(data['Embarked'], prefix = 'port', dummy_na=False)

In [10]:
Embarked_dummies.head()

,port_C,port_Q,port_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [11]:
data = pd.concat([data, Embarked_dummies], axis=1)

In [12]:
data = data.drop(['Embarked'], axis =1 )

In [13]:
data.head() #готовый датасет

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,port_C,port_Q,port_S
0,0,3,0,22.0,1,0,7.2500,0,0,1
1,1,1,1,38.0,1,0,71.2833,1,0,0
2,1,3,1,26.0,0,0,7.9250,0,0,1
3,1,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,0,35.0,0,0,8.0500,0,0,1


In [14]:
X = data
X = X.drop(['Survived'], axis = 1)
y = data['Survived'] # целевой признак 

Построим дерево решений

In [15]:
X.isnull().sum()

Pclass      0
Sex         0
Age       177
SibSp       0
Parch       0
Fare        0
port_C      0
port_Q      0
port_S      0
dtype: int64

In [16]:
X['Age'] = X['Age'].fillna(value = X['Age'].mean()) #заполним все пропущенные значения средним по Age

Сплитанем данные

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [18]:
len(X_train)

596

In [19]:
len(X_test)

295

In [20]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(max_depth = 3)
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [43]:
from sklearn.tree import export_graphviz
export_graphviz(model_tree, feature_names=['Pclassn','Sex','Age','SibSp','Parch','Fare','port_C','port_Q','port_S'], 
                out_file = 'E:\\pyrhon\\progectTEST\\titanic\\tree.dot', filled = True)
#сохраним дерево в формате дот а потом его преобразуем в пнг

In [42]:
!dot -Tpng E:\\pyrhon\\progectTEST\\titanic\\tree.dot -o E:\\pyrhon\\progectTEST\\titanic\\tree.png

In [44]:
y_predict  = model_tree.predict(X_test)

In [45]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))

0.8203389830508474
